In [5]:
import pandas as pd

import app.facade as facade
import app.strategy as strategy
from app.market.stock_market import StockMarket


In [6]:
stock_service = StockMarket()

tudata = facade.TushareData(token='7ba597f7da0b37aa7303298c65dcb17d72ee50ccdd345563b9ded06e',
                            my_path='/Users/ZhaoJunfeng/workspace/python/quant-trader/cmd/tools/sql_data/',
                            db_name='stock_data.db')

In [7]:
cols = ['close', 'open', 'high', 'low']
# 获取沪深全市场A股代码
codes = stock_service.get_codelist()
codes = codes[:1000]

start_date='20200101'
end_date='20220801'

## 策略一：MM趋势+RPS+资金流选股池+每日指标

In [10]:
prices = tudata.sql_adj_data()['close'].dropna(axis=1)
rps = strategy.RPS(prices)
df_rps = rps.date_rps()

# MM趋势
mm_trend=prices.apply(strategy.MM_trend).T
mm_result=mm_trend.query('满足条件==1')

# mm趋势+120日rps>90
mm_rps_result=pd.concat([mm_result,df_rps.query('rps_120>90')],join='inner',axis=1)
# mm_rps_result.sort_values('rps_250',ascending=False)[:10]

# 资金流
code_list=strategy.moneyflow_stock(mm_rps_result.index)
print(f'最近1、3、5、10、20、60日主力资金累计净流入均大于0个股数量：{len(code_list)}')

# mm趋势+RPS+资金流选股
#mm_rps_result.loc[(set(mm_rps_result.index)&set(code_list))]

# 每日指标数据
trade_daily = tudata.sql_all_data(date='20220801')
#单日主力资金净流入
c1=trade_daily['主力净流入']>0
#量比
c2=trade_daily['量比']>1
#市盈率
c3=(0<trade_daily['市盈率'])&(trade_daily['市盈率']<80)
#流通市值大于100亿元
c4=trade_daily['流通市值']/10000>100
#换手率
c5=trade_daily['换手率']>5
#还可以结合市净率、股息率等指标进一步选股
c=c1&c2&c3&c4&c5
#以主力净流入排名，查看前十
daily_result=trade_daily[c]
daily_cols=['trade_date','主力净流入','换手率','量比','市盈率','市净率','市销率','股息率','流通市值']
daily_result=daily_result.set_index('ts_code')[daily_cols]
# daily_result.sort_values('主力净流入',ascending=False)[:10]

# mm趋势+RPS+资金流选股+每日指标
df1=mm_rps_result.loc[code_list][['收盘价','rps_120']]
df2=daily_result.copy()
pd.concat([df1,df2],join='inner',axis=1)

100%|██████████| 254/254 [00:35<00:00,  7.15it/s]


最近1、3、5、10、20、60日主力资金累计净流入均大于0个股数量：14


,收盘价,rps_120,trade_date,主力净流入,换手率,量比,市盈率,市净率,市销率,股息率,流通市值
ts_code,,,,,,,,,,,


KeyboardInterrupt: 